In [10]:
# conda env : freecad_fem_env

import os
import sys

sys.path.append(os.path.join(os.environ["CONDA_PREFIX"], "lib"))

import FreeCAD
import Part
import ObjectsFem
import Mesh
import MeshPart
import Fem
import femmesh.femmesh2mesh
from femtools import ccxtools
import trimesh
import numpy as np
import datetime

from FreecadFemLib import *


In [11]:
#init
doc = App.newDocument("doc")
save_path = "/scratch/aifa/DATA/FCAD_3D/"

In [12]:
trimesh_scene_meshes = []
constraint_scene_meshes = []

In [13]:
doc = document_setup(doc)

load_path = "/scratch/aifa/DATA/FCAD_3D/"
# filename = "_20210517153229478077.obj"#init
# filename = '_20210517163929573370.obj'
obj_file_list = os.listdir(load_path)
rn = np.random.randint(len(obj_file_list))
filename = obj_file_list[rn]
# obj_file_list = ['_20210517163929573370.obj','_20210517174956901483.obj','_20210518001808400542.obj','resultmesh_fcad2.obj']

doc, state0, initMesh, mesh_OK = init_shape(doc, load_path, filename,load_3d=True)
doc = remove_old_mesh_result_mesh(doc,name=initMesh)        
if not mesh_OK: print("XXX  Mesh is not healthy!  XXX")


trimesh_scene_meshes.append(state0)


 ******* _20210518010556946861.obj is loaded ******* 

     +++ Mesh is healthy!  +++ 
shape result mesh is removed...
mesh result mesh is removed...


In [ ]:
for state_indx in range(1,5):

    if not mesh_OK: break
    #------  setup the state-action
    print(f"\n\n-----------\n state - {state_indx}\n-----------\n\n")
    force_position = np.random.randint(3,97)
    print(f"force position : {force_position}")
    
    doc , fem_ok = create_shape_femmesh(doc)
    if not fem_ok: break
    doc , fixed_indx = set_constraint_fixed(doc)
    doc , force_indx = set_constraint_force_placement(doc, force_position=force_position)    
                                            
    if len(fixed_indx)==0 or len(force_indx)==0:
        print("force or fixed constraints are empty... no more analysis will be executed... ")
        break
        
    fixed_scene = add_constraints_to_scene_meshes(doc,fixed_indx,color='blue')
    force_scene = add_constraints_to_scene_meshes(doc,force_indx,color='red')
    constraint_scene_meshes.append(concat_meshes(fixed_scene,force_scene))

    doc = remove_old_mesh_result_mesh(doc)
    doc = remove_old_shape_result_mesh(doc)
    
    doc = set_constraint_force_value(doc, force=3000)
    # run Analysis
    doc, fem_volume = run_analysis(doc)
    if not fem_volume: break
    
#     print("*************")
#     for objs in doc.Objects:
#         print(f">{objs.Name}")
#     print("*************")
    

    #------   create and export result mesh
    doc, out_mesh = create_mesh_from_result(doc)
    if not out_mesh: break
    if not checkMesh(out_mesh): break

    result_trimesh = Meshobj_to_trimesh(out_mesh)
    result_trimesh_decimated = result_trimesh.simplify_quadratic_decimation(2000)
    trimesh_decimated_topology = trimesh_to_mesh_topology(result_trimesh_decimated)
    result_trimesh_decimated = set_trimesh_color(result_trimesh_decimated)

    trimesh_scene_meshes.append(result_trimesh_decimated)
    #view_current_state(doc, result_trimesh_decimated)

    now = str(datetime.datetime.now())
    sav = "_"+now.replace("-","").replace(" ","").replace(":","").split(".")[0]

    saved_filename = f"mAct_{sav}.obj"
    save_trimesh(result_trimesh_decimated,save_path,saved_filename)
    #export_result_mesh(doc,state_indx)
    
    #------   prepare shape for next state
    doc = remove_old_femmesh(doc)
    doc = clear_constraints(doc)
    doc = remove_old_solid(doc)

    doc = create_shape_from_mesh(doc,trimesh_decimated_topology)
    doc, solid = convert_to_solid(doc) 

print("+++++ Done! +++++")

In [6]:
# mesh_scene = generate_complete_meshes_scene(doc,trimesh_scene_meshes)
# mesh_scene.show()

In [7]:
# print(len(constraint_scene_meshes),len(trimesh_scene_meshes))

# n = np.random.randint(len(constraint_scene_meshes))
# print(f"the [state{n} - action{n}] (before running the analysis)")
# state_scene = []
# state_scene.append(trimesh_scene_meshes[n])
# state_scene.append(constraint_scene_meshes[n])

# _scene = generate_complete_meshes_scene(doc,state_scene)
# _scene.show()

In [8]:
# s = scene_view_normals(result_trimesh_decimated)
# s.show()

In [9]:
### save_image() has problem with opengl (and jupyter)

# from PIL import Image
# import io 
# image = np.array(Image.open(io.BytesIO(trimesh.Scene(pnt).save_image(resolution=(200, 200)))))

# trimesh.Scene(pnt).show(line_settings={'point_size':130})